Install LlamaIndex Libraries

In [2]:
'''
%pip install llama-index-llms-openai pandas[jinja2] spacy
!pip install llama-index-llms-langchain
'''

'\n%pip install llama-index-llms-openai pandas[jinja2] spacy\n!pip install llama-index-llms-langchain\n'

In [3]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    Settings,
    Response
)
from llama_index.core.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator
)
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.core.node_parser import SentenceSplitter

import os
import time
import nest_asyncio

import pandas as pd

In [16]:
nest_asyncio.apply()
pd.set_option("display.max_colwidth", 0)

Set up LLM for question generation

In [10]:
# Definir el modelo local (LMStudio)
from llama_index.llms.lmstudio import LMStudio

llm = LMStudio(
    model_name="SomeModel",
    base_url="http://localhost:1234/v1",
    temperature=0.7,)

Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [40]:
# Definir el modelo por API (Groq)

# pip install llama-index-llms-groq
from llama_index.llms.groq import Groq

# Set up the Groq class with the required model and API key
os.environ["GROQ_API_KEY"] = "XXX" # Definir la API Key de Groq
api_key = os.getenv("GROQ_API_KEY", "NotFound") # Obtener la API Key de las variables de entorno
print(api_key)

llm = Groq(model="llama3-70b-8192", api_key=api_key)

XXX


Load the documents

In [31]:
# Get an example document
documents = SimpleDirectoryReader(input_files=["18_EstructurasDeDatos_4196.json"]).load_data()
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.llm = llm

# Create a vector index
splitter = SentenceSplitter(chunk_size=512)
vector_index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter]
)

In [36]:
eval_documents = documents[:3]
data_generator = DatasetGenerator.from_documents(eval_documents)
eval_questions = data_generator.generate_questions_from_nodes()

print(eval_questions)

c:\Users\Luis Alejandro\AppData\Local\Programs\Python\Python310\lib\site-packages\llama_index\core\evaluation\dataset_generation.py:200: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


['Here are 10 questions based on the provided context information:', 'What is the title of the course described in the JSON file?', 'a) Estructuras de Datos', 'b) Programación Avanzada', 'c) Lógica y Matemáticas Discretas', 'd) Fundamentos de Matemáticas II', 'Answer: a) Estructuras de Datos', 'What is the code of the course described in the JSON file?', 'a) 001297', 'b) 4196', 'c) 2430', 'd) 2024', 'Answer: b) 4196', 'What is the number of credits for the course described in the JSON file?', 'a) 2', 'b) 3', 'c) 4', 'd) 5', 'Answer: b) 3', 'What is the access control type for the file described in the context information?', 'a) Público', 'b) Privado', 'c) Restringido', 'd) Confidencial', 'Answer: b) Privado', 'What is the last updated date mentioned in the JSON file?', 'a) 2024-08-20', 'b) 2024-08-21', 'c) 2024-08-25', 'd) 2024-08-30', 'Answer: a) 2024-08-20', 'What is the description of the course described in the JSON file?', 'a) En la mayoría de los ambientes reales de desarrollo de

c:\Users\Luis Alejandro\AppData\Local\Programs\Python\Python310\lib\site-packages\llama_index\core\evaluation\dataset_generation.py:296: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


Set up the evaluator

In [37]:
from llama_index.core.evaluation import EvaluationResult

# define jupyter display function
def display_eval_df(response: Response, eval_result: EvaluationResult) -> None:
    if response.source_nodes == []:
        print("no response!")
        return
    eval_df = pd.DataFrame(
        {
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
            "Reasoning": eval_result.feedback,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [38]:
evaluator = FaithfulnessEvaluator(llm=llm)

Query the vector DB and evaluate the answers

In [39]:
vector_index = VectorStoreIndex.from_documents(
    eval_documents
)
query_engine = vector_index.as_query_engine(llm)

response_vector = query_engine.query("Which are the contents of Estructuras de datos?")
eval_result = evaluator.evaluate_response(response=response_vector)
display_eval_df(response_vector, eval_result)

,Response,Source,Evaluation Result,Reasoning
0,"The contents of Estructuras de datos are: Bases de complejidad algorítmica, Estructuras lineales, Estructuras recurrentes, and Estructuras no lineales.","{ ""version"": ""1.0"", ""last_updated"": ""2024-08-20"", ""title"": ""18_EstructurasDeDatos_4196"", ""description"": ""Syllabus de la materia 'EstructurasDeDatos' con código '4196'"", ""access_control"": ""privado"", ""content"": { ""Nombre Corto de la Asignatura"": ""Estructuras de Datos"", ""Nombre Largo de la Asignatura"": ""Estructuras de Datos"", ""Código de la asignatura"": ""4196"", ""Grado"": ""Pregrado"", ""Descripción"": ""En la mayoría de los ambientes reales de desarrollo de software se procesan grandes cantidades de información, que está representada en datos que tienen sentido, secuencia y magnitud. Entonces, los correctos diseños e implementaciones de estructuras de datos y algoritmos son vitales para el funcionamiento adecuado de los sistemas de cómputo y almacenamiento en la vida real.\n\nEste curso se centra en el estudio de las estructuras contenedoras de datos básicas y los algoritmos relacionados con ellas, sin perder de vista su ...",Pass,YES
